
# Making a Simple Bigram model with Romeo and Juliet Book


In [12]:
import torch
print(torch.__version__)

2.4.0+cpu



## 1) read the text file with data and store it in a variable

    the contents of the book was stored in a `data.txt file` and all it['s content is read and copied to a variable named text.
    the length and the first 300 characters was printed to test if it was working and once confirmed moved on the second part.

In [13]:
with open('data.txt', 'r', encoding='utf-8') as file:
    text = file.read()  
print(len(text))
print(text[:300])

142466
﻿THE TRAGEDY OF ROMEO AND JULIET

by William Shakespeare




Contents

THE PROLOGUE.

ACT I
Scene I. A public place.
Scene II. A Street.
Scene III. Room in Capulet’s House.
Scene IV. A Street.
Scene V. A Hall in Capulet’s House.

ACT II
CHORUS.
Scene I. An open place adjoining Capulet’s Garden.
Scen



## 2) make a sorted set of all the characters in the dataset

    a set of all the unique characters in the data was taken using the `set()` method and the result was passed into the `sorted()` method and copied   to the `chars` variable, The result is printed for testing and visualisation purposes. This helps us have a vocabulary of sorts to work with.

In [14]:
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '&', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', '—', '‘', '’', '“', '”', '\ufeff']
71



 ### why do the above ? 

  this lets us work with tokenizers which comes with encoders and decoders, encoders can help us convert all the 71 characters to numbers. we will be making a character level tokenizer below


## 3) make a basic character-level tokenizer


In order to make the character level tokenizer two dictionaries are first made. **'string_to_integer = { ch:i for i,ch in enumerate(chars) }'** enumerate over each character in the array and assign it an unique number making the first dictionary and reverse for integer to string making the second dictionary .

**e.g. :** if  'chars = ['a', 'b', 'c']' then these will be the values of the two dictionaries :
string_to_integer = {'a': 0, 'b': 1, 'c': 2}
integer_to_string = {0: 'a', 1: 'b', 2: 'c'}

the encoder lambda function takes a string s and converts it to a list of integers. For each character c in the string s, it looks up its integer value using the string_to_integer dictionary. The result is a list of integers corresponding to each character.

the decoder lambda function takes a list of integers l and converts it back to a string. For each integer i in the list l, it looks up the corresponding character using the integer_to_string dictionary. It then joins these characters into a single string.

**another e.g:** 

encoder('abc')  # e.g. Output: [0, 1, 2]

decoder([0, 1, 2])  # e.g. Output: 'abc'

these two methods combied helps have the functionality of a basic character-level tokenizer


In [15]:
string_to_integer = { ch:i for i,ch in enumerate(chars) }
integer_to_string = { i:ch for i,ch in enumerate(chars) }
encoder = lambda s: [string_to_integer[c] for c in s]
decoder = lambda l: ''.join([integer_to_string[i] for i in l])

encoded_yungting = encoder('yungting')
print(encoded_yungting)
decoded_yungting = decoder(encoded_yungting)
print(decoded_yungting)

[62, 58, 51, 44, 57, 46, 51, 44]
yungting



### <ins>More info on tokenizers</ins>

 ### Types of tokenizers 

 #### 1) Character-level tokenizer

- Small Vocabulary but large amount of chcaracters to work with.For example, for the romeo and julirt data we are using we have 142466 characters with a vocabulary of 71 unique characters. There are more characters to encode and decode.


 #### 2) word-level tokenizer 

- Very large Vocabulary but smaller amount of words to work with.The vocabulary can get very big especially if there is multiple languages.Less words to encode and decode but very large dictionary

#### 3) subword-tokenizer

- Somewhere inbetween character-level tokenizer and word-level tokenizer


##### it is very important to be efficient with data when working with language models
         

## 4) Working with tensors 

#### 1) What are tensors
- tensors are the main data structure we will be working with , pytorch does most of the math for it for us

In [16]:
x = torch.empty(3, 4)
print(type(x))
print(x)

<class 'torch.Tensor'>
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


- as seen above x is of the type Tensor and it is 3 rows and 4 columns and is empty
- 1 dimensional tensor is usally called a vector and is reffered to as such
- 2 dimensional tensor is usually called a matrix and referred to as such
- 3 dimensional and above tensors are usually just referred to as tensors
- as per definitions above we can refer to x as a matrix as it is 2 dimensional
- x i populated with 32 bit floats by defaults , if you see random values it is just the last values from memory
- We use tensors over arrays and stuff as it is easier for pytorch to work with it especially when reshaping, changing dimensionality,multiplying, doing dot products, and so on

#### 2) Working with tensors and our enncoder we made 


- we will use our encoder to encode the entire 'Romeo and Juliet' novel into numbers then we will use torch tensor to make a tensor with the numbers returned from the encoder we made , while making the tensor we can manually set the data type to be sure that the tensor we are making is gonna use the mentioned data type of long
- the first 120 characters will be printed out as a test

In [21]:
data = torch.tensor(encoder(text), dtype=torch.long)
print(data[:120])

tensor([70, 29, 17, 14,  1, 29, 27, 10, 16, 14, 13, 33,  1, 24, 15,  1, 27, 24,
        22, 14, 24,  1, 10, 23, 13,  1, 19, 30, 21, 18, 14, 29,  0,  0, 39, 62,
         1, 32, 46, 49, 49, 46, 38, 50,  1, 28, 45, 38, 48, 42, 56, 53, 42, 38,
        55, 42,  0,  0,  0,  0,  0, 12, 52, 51, 57, 42, 51, 57, 56,  0,  0, 29,
        17, 14,  1, 25, 27, 24, 21, 24, 16, 30, 14,  6,  0,  0, 10, 12, 29,  1,
        18,  0, 28, 40, 42, 51, 42,  1, 18,  6,  1, 10,  1, 53, 58, 39, 49, 46,
        40,  1, 53, 49, 38, 40, 42,  6,  0, 28, 40, 42])


## 5) Training and Validation splits

-  #### i) why do we need to split our data into training and validation splits ?
     - we split it this way as